In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Organising data into RAP, test and validation sets
os.chdir('/content/drive/MyDrive/RAP')
if os.path.isdir('RAP/open') is False:
  os.makedirs('/content/drive/MyDrive/RAP/train/open')
  os.makedirs('/content/drive/MyDrive/RAP/train/close')
  os.makedirs('/content/drive/MyDrive/RAP/valid/open')
  os.makedirs('/content/drive/MyDrive/RAP/valid/close')
  os.makedirs('/content/drive/MyDrive/RAP/test/close')
  os.makedirs('/content/drive/MyDrive/RAP/test/open')

  for c in random.sample(glob.glob('close*'), 500):
      shutil.move(c, '/content/drive/MyDrive/RAP/train/close')
  for c in random.sample(glob.glob('open*'), 500):
      shutil.move(c, '/content/drive/MyDrive/RAP/train/open')
  for c in random.sample(glob.glob('close*'),100):
      shutil.move(c, '/content/drive/MyDrive/RAP/valid/close')
  for c in random.sample(glob.glob('open*'), 100):
      shutil.move(c, '/content/drive/MyDrive/RAP/valid/open')
  for c in random.sample(glob.glob('close*'), 50):
      shutil.move(c, '/content/drive/MyDrive/RAP/test/close')
  for c in random.sample(glob.glob('open*'), 50):
      shutil.move(c, '/content/drive/MyDrive/RAP/test/open')

os.chdir('../../')



In [ ]:
train_path = '/content/drive/MyDrive/RAP/train'
test_path = '/content/drive/MyDrive/RAP/test'
valid_path = '/content/drive/MyDrive/RAP/valid'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['close', 'open'], batch_size=10)
valid_batches =  ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['close', 'open'], batch_size=10)
test_batches =  ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['close', 'open'], batch_size=10, shuffle= False)    


In [ ]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100

assert train_batches.num_classes == valid_batches.num_classes==test_batches.num_classes == 2

In [ ]:
# this function will plot the image in a grid
def plotImages(images_arr):
    fig, axes = plt.subplot(1, 10, figsize=(20,20))
    axes=axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
model = Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)),
                    MaxPool2D(pool_size=(2,2), strides=2),
                    Conv2D(filters=64,kernel_size=(3,3), activation='relu',padding='same'),
                    MaxPool2D(pool_size=(2,2),strides=2),
                    Flatten(),
                    Dense(units=2,activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)